In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/39.OEMPartsOnline'
else:
    print('fuck')

path_root

Mounted at /content/drive


'./drive/MyDrive/code/03.crawler/39.OEMPartsOnline'

In [2]:
import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append(os.path.join(path_root, '../00.Tools'))
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import json

# = = =  = = =  = = =

import itertools

# = = =  = = =  = = =

import re
import html as html_lib

In [3]:
directory_in = os.path.join(path_root, './file/2.url')
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'crawler_1.xlsx'

In [4]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：450



,No,Url
0,1,https://toyota.oempartsonline.com/oem-parts/to...
1,2,https://toyota.oempartsonline.com/oem-parts/to...
2,3,https://toyota.oempartsonline.com/oem-parts/to...
3,4,https://toyota.oempartsonline.com/oem-parts/to...
4,5,https://toyota.oempartsonline.com/oem-parts/to...
...,...,...
445,446,https://toyota.oempartsonline.com/oem-parts/to...
446,447,https://toyota.oempartsonline.com/oem-parts/to...
447,448,https://toyota.oempartsonline.com/oem-parts/to...
448,449,https://toyota.oempartsonline.com/oem-parts/to...


In [5]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                     1
 Url    https://toyota.oempartsonline.com/oem-parts/to...
 Name: 0, dtype: object)

In [6]:
import time

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_request = crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            dict_ = json.loads(html.xpath('//script[@type="application/json"]/text()')[0])
            if dict_:
                with open(os.path.join(path_root, './part.html'), 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except requests.exceptions.ProxyError as e:
        if '456 CN Client Forbidden' in str(e):
            print('Long live the People\'s Republic of China!')
            break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'in_rp_catalog': True,
 'source': 'product',
 'sku': '17893-0V010',
 'sku_stripped': '178930v010',
 'catalog': 'toyota',
 'price': 25.38,
 'msrp': 33.76,
 'images': [{'main': {'url': '//dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/d655e6389db16fdce6cf83a47e0f3b4f.png',
    'width': 640,
    'height': 471.09278350515467},
   'preview': {'url': '//dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/f5d4f18466f181fb5f86b441537bc746.png',
    'width': 300,
    'height': 220.82474226804126},
   'thumb': {'url': '//dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/8256d42c091e2fb2b4b9e45418185c80.png',
    'width': 150,
    'height': 110.41237113402063},
   'type': 'PART',
   'image_id': 'd655e6389db16fdce6cf83a47e0f3b4f',
   'alt_text': '',
   'is_primary': False,
   'is_visible': True,
   'filename': '',
   'has_watermark': 'Unknown',
   'md5': '',
   'perceptual_hash': '',
   'product_sku_stripped': None,
   'hotspot': {'refe

In [7]:
dict_src, list_img = {}, dict_['images']
for i, img in enumerate(list_img):
    dict_src[str(i)] = f'''https:{img['main']['url'].strip()}''' if img['main']['url'].strip().startswith('//') else img['main']['url'].strip()

dict_src

{'0': 'https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/d655e6389db16fdce6cf83a47e0f3b4f.png',
 '1': 'https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/b037b8d425c3063f7f3a16012e5d5424.png',
 '2': 'https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/77b4bb9f4120f0b01ec667daced5d2cc.png'}

In [8]:
list_dict_vehicle, list_dict_fitment = [], dict_['fitment']
for dict_fitment in list_dict_fitment:
    dict_fitment.pop('idx')
    list_key = list(dict_fitment.keys())
    list_value = [dict_fitment[key] if type(dict_fitment[key]) == list else [dict_fitment[key]] for key in list_key]

    product_combination = itertools.product(*list_value)
    for combination in product_combination:
        list_dict_vehicle.append(dict(zip(list_key, combination)))

list_dict_vehicle

[{'year': '2015',
  'make': 'Toyota',
  'model': 'Venza',
  'trims': 'LE',
  'engines': '2.7L L4 - Gas'},
 {'year': '2015',
  'make': 'Toyota',
  'model': 'Venza',
  'trims': 'XLE',
  'engines': '2.7L L4 - Gas'},
 {'year': '2014',
  'make': 'Toyota',
  'model': 'Venza',
  'trims': 'LE',
  'engines': '2.7L L4 - Gas'},
 {'year': '2014',
  'make': 'Toyota',
  'model': 'Venza',
  'trims': 'XLE',
  'engines': '2.7L L4 - Gas'},
 {'year': '2013',
  'make': 'Toyota',
  'model': 'Venza',
  'trims': 'LE',
  'engines': '2.7L L4 - Gas'},
 {'year': '2013',
  'make': 'Toyota',
  'model': 'Venza',
  'trims': 'XLE',
  'engines': '2.7L L4 - Gas'},
 {'year': '2012',
  'make': 'Toyota',
  'model': 'Venza',
  'trims': 'LE',
  'engines': '2.7L L4 - Gas'},
 {'year': '2012',
  'make': 'Toyota',
  'model': 'Venza',
  'trims': 'XLE',
  'engines': '2.7L L4 - Gas'},
 {'year': '2011',
  'make': 'Toyota',
  'model': 'Camry',
  'trims': 'Base',
  'engines': '2.5L L4 - Gas'},
 {'year': '2011',
  'make': 'Toyota',
  

In [9]:
subtitle = html.xpath('//div[@class="page-builder-layout-module desktop-only"]/descendant::p[@class="product-subtitle"]/text()')[0].strip() if html.xpath('//div[@class="page-builder-layout-module desktop-only"]/descendant::p[@class="product-subtitle"]/text()') else ''

subtitle

'2009-2015 Toyota - 178930v010'

In [10]:
dict_key, list_li = {}, html.xpath('//h2[@class="product-details-title"]/following-sibling::ul[@class="field-list"][1]/li')
for li in list_li:
    key = li.xpath('./span[@class="list-label"]')[0]
    key = etree.tostring(key).decode('utf-8')
    key = re.sub(r'<.*?>', '', key)
    key = re.sub(r' +', ' ', key)
    key = html_lib.unescape(key)
    key = key.strip().removesuffix(':').strip()
    if key == 'Brand':
        continue
    elif key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_detail, dict_count = {}, {}
for i, li in enumerate(list_li):
    key = li.xpath('./span[@class="list-label"]')[0]
    key = etree.tostring(key).decode('utf-8')
    key = re.sub(r'<.*?>', '', key)
    key = re.sub(r' +', ' ', key)
    key = html_lib.unescape(key)
    key = key.strip().removesuffix(':').strip()
    if key == 'Brand':
        list_src = li.xpath('./span[@class="list-value"]/div[@class="part-manufacturer"]/div[@class="manufacturer-logos"]/img/@src')
        list_src = ['.'.join(src.split('/')[-1].split('.')[:-1]).title().strip() for src in list_src]
        dict_detail[str(i)] = {key: ';'.join(list_src)}
    else:
        value = li.xpath('./span[contains(@class, "list-value")]')[0]
        value = etree.tostring(value).decode('utf-8')
        value = re.sub(r'<.*?>', '', value)
        value = re.sub(r' +', ' ', value)
        value = html_lib.unescape(value)
        if dict_key[key] == 1:
            dict_detail[str(i)] = {key: value.strip()}
        else:
            if key in dict_count:
                dict_count[key] += 1
            else:
                dict_count[key] = 1
            dict_detail[str(i)] = {f'{key}-{dict_count[key]}': value.strip()}

dict_detail

{'0': {'Brand': 'Toyota;Scion'},
 '1': {'SKU': '17893-0V010'},
 '2': {'Positions': 'Lower'},
 '3': {'Other Names': 'Resonator Intake Ai'},
 '4': {'Description': '2.7L, lower. 2.5L, #1, us built.'},
 '5': {'Condition': 'New'}}

In [11]:
dict_correct_temp = {'No': crawler_series['No'],
                     'Catalog': dict_['catalog'].title().strip(),
                     'Part Number': dict_['sku'].strip(),
                     'Title': dict_['title'].strip(),
                     'Subtitle': subtitle,
                     'Price': dict_['price'],
                     'Picture': '',
                     'Url': crawler_series['Url'],
                     'Json_Src': json.dumps(dict_src),
                     'Json_Detail': json.dumps(dict_detail),
                     'JOIN_CPN': f'''{dict_['catalog'].title().strip()};{dict_['sku'].strip()}'''}

dict_correct_temp

{'No': 1,
 'Catalog': 'Toyota',
 'Part Number': '17893-0V010',
 'Title': 'Resonator - Toyota (17893-0V010)',
 'Subtitle': '2009-2015 Toyota - 178930v010',
 'Price': 25.38,
 'Picture': '',
 'Url': 'https://toyota.oempartsonline.com/oem-parts/toyota-resonator-178930v010',
 'Json_Src': '{"0": "https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/d655e6389db16fdce6cf83a47e0f3b4f.png", "1": "https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/b037b8d425c3063f7f3a16012e5d5424.png", "2": "https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/77b4bb9f4120f0b01ec667daced5d2cc.png"}',
 'Json_Detail': '{"0": {"Brand": "Toyota;Scion"}, "1": {"SKU": "17893-0V010"}, "2": {"Positions": "Lower"}, "3": {"Other Names": "Resonator Intake Ai"}, "4": {"Description": "2.7L, lower. 2.5L, #1, us built."}, "5": {"Condition": "New"}}',
 'Json_CPN': 'Toyota;17893-0V010'}

In [12]:
list_dict_correct_temp = [{**dict_correct_temp, **dict_vehicle} for dict_vehicle in list_dict_vehicle]

list_dict_correct_temp

[{'No': 1,
  'Catalog': 'Toyota',
  'Part Number': '17893-0V010',
  'Title': 'Resonator - Toyota (17893-0V010)',
  'Subtitle': '2009-2015 Toyota - 178930v010',
  'Price': 25.38,
  'Picture': '',
  'Url': 'https://toyota.oempartsonline.com/oem-parts/toyota-resonator-178930v010',
  'Json_Src': '{"0": "https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/d655e6389db16fdce6cf83a47e0f3b4f.png", "1": "https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/b037b8d425c3063f7f3a16012e5d5424.png", "2": "https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/77b4bb9f4120f0b01ec667daced5d2cc.png"}',
  'Json_Detail': '{"0": {"Brand": "Toyota;Scion"}, "1": {"SKU": "17893-0V010"}, "2": {"Positions": "Lower"}, "3": {"Other Names": "Resonator Intake Ai"}, "4": {"Description": "2.7L, lower. 2.5L, #1, us built."}, "5": {"Condition": "New"}}',
  'Json_CPN': 'Toyota;17893-0V010',
  'year': '2015',
  'make': 'Toyota',
  'model': 'Venza',

In [13]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'No': 1,
  'Catalog': 'Toyota',
  'Part Number': '17893-0V010',
  'Title': 'Resonator - Toyota (17893-0V010)',
  'Subtitle': '2009-2015 Toyota - 178930v010',
  'Price': 25.38,
  'Picture': '',
  'Url': 'https://toyota.oempartsonline.com/oem-parts/toyota-resonator-178930v010',
  'Json_Src': '{"0": "https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/d655e6389db16fdce6cf83a47e0f3b4f.png", "1": "https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/b037b8d425c3063f7f3a16012e5d5424.png", "2": "https://dz310nzuyimx0.cloudfront.net/strapr1/85764bf6209c40a8d958e3d339261b97/77b4bb9f4120f0b01ec667daced5d2cc.png"}',
  'Json_Detail': '{"0": {"Brand": "Toyota;Scion"}, "1": {"SKU": "17893-0V010"}, "2": {"Positions": "Lower"}, "3": {"Other Names": "Resonator Intake Ai"}, "4": {"Description": "2.7L, lower. 2.5L, #1, us built."}, "5": {"Condition": "New"}}',
  'Json_CPN': 'Toyota;17893-0V010',
  'year': '2015',
  'make': 'Toyota',
  'model': 'Venza',

In [14]:
crawler_status = True

crawler_status

True

In [15]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://toyota.oempartsonline.com/oem-parts/to...
 Request_Url    https://toyota.oempartsonline.com/oem-parts/to...
 Name: 0, dtype: object]

In [16]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct)
    df_correct['year'] = df_correct['year'].astype(int)
    df_correct = df_correct.sort_values(by=['No', 'Part Number', 'make', 'model', 'trims', 'engines', 'year'],
                                        ascending=[True, True, True, True, True, True, False],
                                        ignore_index=True)
    df_correct.to_excel(os.path.join(path_root, './test_part.xlsx'), index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Catalog,Part Number,Title,Subtitle,Price,Picture,Url,Json_Src,Json_Detail,Json_CPN,year,make,model,trims,engines
0,1,Toyota,17893-0V010,Resonator - Toyota (17893-0V010),2009-2015 Toyota - 178930v010,25.38,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://dz310nzuyimx0.cloudfront.net/st...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;17893-0V010,2011,Toyota,Camry,Base,2.5L L4 - Gas
1,1,Toyota,17893-0V010,Resonator - Toyota (17893-0V010),2009-2015 Toyota - 178930v010,25.38,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://dz310nzuyimx0.cloudfront.net/st...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;17893-0V010,2011,Toyota,Camry,LE,2.5L L4 - Gas
2,1,Toyota,17893-0V010,Resonator - Toyota (17893-0V010),2009-2015 Toyota - 178930v010,25.38,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://dz310nzuyimx0.cloudfront.net/st...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;17893-0V010,2011,Toyota,Camry,SE,2.5L L4 - Gas
3,1,Toyota,17893-0V010,Resonator - Toyota (17893-0V010),2009-2015 Toyota - 178930v010,25.38,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://dz310nzuyimx0.cloudfront.net/st...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;17893-0V010,2011,Toyota,Camry,XLE,2.5L L4 - Gas
4,1,Toyota,17893-0V010,Resonator - Toyota (17893-0V010),2009-2015 Toyota - 178930v010,25.38,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://dz310nzuyimx0.cloudfront.net/st...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;17893-0V010,2010,Toyota,Camry,Base,2.5L L4 - Gas
5,1,Toyota,17893-0V010,Resonator - Toyota (17893-0V010),2009-2015 Toyota - 178930v010,25.38,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://dz310nzuyimx0.cloudfront.net/st...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;17893-0V010,2010,Toyota,Camry,LE,2.5L L4 - Gas
6,1,Toyota,17893-0V010,Resonator - Toyota (17893-0V010),2009-2015 Toyota - 178930v010,25.38,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://dz310nzuyimx0.cloudfront.net/st...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;17893-0V010,2010,Toyota,Camry,SE,2.5L L4 - Gas
7,1,Toyota,17893-0V010,Resonator - Toyota (17893-0V010),2009-2015 Toyota - 178930v010,25.38,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://dz310nzuyimx0.cloudfront.net/st...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;17893-0V010,2010,Toyota,Camry,XLE,2.5L L4 - Gas
8,1,Toyota,17893-0V010,Resonator - Toyota (17893-0V010),2009-2015 Toyota - 178930v010,25.38,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://dz310nzuyimx0.cloudfront.net/st...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;17893-0V010,2015,Toyota,Venza,LE,2.7L L4 - Gas
9,1,Toyota,17893-0V010,Resonator - Toyota (17893-0V010),2009-2015 Toyota - 178930v010,25.38,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://dz310nzuyimx0.cloudfront.net/st...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;17893-0V010,2015,Toyota,Venza,XLE,2.7L L4 - Gas


In [17]:
print('Output Finished !')
print()

Output Finished !



In [18]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel(os.path.join(path_root, './test_part_error.xlsx'), index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,Request_Url
0,1,https://toyota.oempartsonline.com/oem-parts/to...,https://toyota.oempartsonline.com/oem-parts/to...


In [19]:
print('Done ~')

Done ~
